In [ ]:
pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━

In [ ]:
!unzip recipe_model_output_phi2.zip

Archive:  recipe_model_output_phi2.zip
Written using ZipTricks 5.6.0
   creating: checkpoint-1200/
  inflating: checkpoint-1200/README.md  
  inflating: checkpoint-1200/adapter_config.json  
  inflating: checkpoint-1200/adapter_model.safetensors  
  inflating: checkpoint-1200/optimizer.pt  
  inflating: checkpoint-1200/rng_state.pth  
  inflating: checkpoint-1200/scheduler.pt  
  inflating: checkpoint-1200/trainer_state.json  
  inflating: checkpoint-1200/training_args.bin  
   creating: checkpoint-1447/
  inflating: checkpoint-1447/README.md  
  inflating: checkpoint-1447/adapter_config.json  
  inflating: checkpoint-1447/adapter_model.safetensors  
  inflating: checkpoint-1447/optimizer.pt  
  inflating: checkpoint-1447/rng_state.pth  
  inflating: checkpoint-1447/scheduler.pt  
  inflating: checkpoint-1447/trainer_state.json  
  inflating: checkpoint-1447/training_args.bin  
   creating: checkpoint-400/
  inflating: checkpoint-400/README.md  
  inflating: checkpoint-400/adapter_conf

In [ ]:
import torch
import json
import pandas as pd
import ast
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
from peft import PeftModel, PeftConfig
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceEndpoint
import re
import logging
from sklearn.model_selection import train_test_split

MODEL_NAME = "microsoft/phi-2"
MAX_LENGTH = 512
NUM_SAMPLES = 1
FINETUNED_PATH = "final_model"
EMBEDDINGS_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
ALLERGEN_DB_PATH = "allergen_substitutes.json"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

def load_recipes():
    print("Loading recipe dataset...")
    recipes = pd.read_csv('RAW_recipes.csv', encoding='utf8')
    recipes = recipes[['name', 'ingredients', 'steps']]
    recipes['name'] = recipes['name'].apply(lambda x: [x] if isinstance(x, str) else x)
    recipes['steps'] = recipes['steps'].apply(ast.literal_eval)
    recipes['ingredients'] = recipes['ingredients'].apply(ast.literal_eval)
    subsample_size = len(recipes)
    recipes = recipes.sample(n=subsample_size, random_state=42)
    logging.info(f"Using a subsample of {len(recipes)} recipes")

    def format_recipe(row):
        try:
            name = row['name']
            if isinstance(name, list):
                name = ' '.join(map(str, name))  # Join list elements into a single string
            else:
                name = str(name)  # If it's not a list, just convert it to a string

            # Handle 'ingredients' field (ensure it's a list or convert to a string if it's not)
            ingredients = row['ingredients']
            if isinstance(ingredients, list):
                ingredients = ', '.join(map(str, ingredients))  # Join list elements into a single string
            else:
                ingredients = str(ingredients)  # If it's not a list, just convert it to a string

            # Handle 'steps' field (ensure it's a list or convert to a string if it's not)
            steps = row['steps']
            if isinstance(steps, list):
                steps = '\n'.join(map(str, steps))  # Join list elements into a single string with newlines
            else:
                steps = str(steps)  # If it's not a list, just convert it to a string

            # Format the recipe as a string
            prompt = f"Name: {name}\nIngredients: {ingredients}\nInstructions:"
            completion = steps  # Instructions are joined by newlines

            return prompt + completion
        except Exception as e:
            logging.warning(f"Error processing recipe: {str(e)}")
            return None

    formatted_data = recipes.apply(format_recipe, axis=1)
    formatted_data = formatted_data[formatted_data.notna()].tolist()
    copy_formatted_data = formatted_data.copy()
    train_val_text, test_texts = train_test_split(copy_formatted_data, test_size=0.1, random_state=42)
    train_texts, val_texts = train_test_split(train_val_text, test_size=1/9, random_state=42)

    logging.info(f"Split dataset into {len(train_texts)} training, {len(val_texts)} validation samples and {len(test_texts)} test samples")
    return train_texts, val_texts, test_texts

def load_allergen_database():
    """Load and prepare the allergen substitution database for RAG"""
    with open(ALLERGEN_DB_PATH, 'r') as f:
        allergen_data = json.load(f)

    documents = []
    for allergen, info in allergen_data.items():
        doc = f"Allergen: {allergen}\nSubstitutes: {', '.join(info['substitutes'])}\nNotes: {info['notes']}"
        documents.append(doc)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
    )
    texts = text_splitter.create_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDINGS_MODEL)
    vectorstore = FAISS.from_documents(texts, embeddings)

    return vectorstore

def substitute_allergens(recipe_text, vectorstore,ingredients):
    """Use RAG to identify and substitute allergens in the recipe"""
    #ingredients_match = re.search(r"Ingredients:(.*?)Instructions:", recipe_text, re.DOTALL)
    ingredients_list=ingredients.split(', ')
    ingredients_match=[]
    for ingredient in ingredients_list:
      if ingredient in recipe_text:
        ingredients_match.append(ingredient)
    #ingredients_text = ingredients_match.group(1).strip()
    #ingredients_list = [i.strip() for i in ingredients_text.split('\n') if i.strip()]
    print(ingredients)
    if not ingredients_match:
      return recipe_text,[]

    substitutions_made = []
    modified_recipe = recipe_text

    for ingredient in ingredients_match:
        results = vectorstore.similarity_search(
            f"What are substitutes for {ingredient}?",
            k=1
        )

        if results and "Substitutes:" in results[0].page_content:
            substitute_info = results[0].page_content.split("Substitutes:")[1].split("Notes:")[0].strip()
            substitutes = substitute_info.split(', ')[0]

            modified_recipe = modified_recipe.replace(ingredient, f"{substitutes} (substitute for {ingredient})")
            substitutions_made.append({
                "original": ingredient,
                "substitute": substitutes
            })

    return modified_recipe, substitutions_made

def generate_recipe(model, tokenizer, ingredients, name, vectorstore):
    prompt = f"""You are an expert chef and recipe writer. Given a recipe name and a list of ingredients, create a high-quality, detailed recipe.

Create a detailed recipe for: {name}
Using these ingredients: {ingredients}

Recipe:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_LENGTH)
    inputs = {k: v.to(device=model.device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.75,
        top_p=0.8,
        do_sample=True,
        no_repeat_ngram_size=4,
        repetition_penalty=1.3,
        pad_token_id=tokenizer.pad_token_id
    )

    generated_recipe = tokenizer.decode(outputs[0], skip_special_tokens=True)
    original_recipe = generated_recipe.replace(prompt, "").strip()

    #print("\n=== Original Recipe ===")
    #print(original_recipe)

    modified_recipe, substitutions = substitute_allergens(original_recipe, vectorstore,ingredients)

    if substitutions:
        print("\n=== Allergen Substitutions Made ===")
        for sub in substitutions:
            print(f"Replaced '{sub['original']}' with '{sub['substitute']}'")

    #print("\n=== Modified Recipe ===")
    #print(modified_recipe)

    return {
        "original_recipe": original_recipe,
        "modified_recipe": modified_recipe,
        "substitutions": substitutions
    }

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    print("Loading model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
    model.to(device)

    print("Loading PEFT model...")
    finetuned_model = PeftModel.from_pretrained(model, FINETUNED_PATH)
    finetuned_model.eval()


    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.pad_token_id

    print("Loading allergen database and creating vector store...")
    vectorstore = load_allergen_database()
    train_text,val_text,test_text=load_recipes()
    recipe_data =test_text[:500]

    print(f"Generating {NUM_SAMPLES} recipes with allergen substitutions...")
    recipes = []
    for recipe in tqdm(recipe_data):
        parts = recipe.split("\nInstructions:")
        header = parts[0].split("\nIngredients: ")
        name = header[0].replace("Name: ", "")
        ingredients = header[1]
        reference_instructions = parts[1] if len(parts) > 1 else ""
        print("\n" + "="*50)
        print(f"Generating recipe for: {name}")
        print("="*50)
        generated_recipe = generate_recipe(finetuned_model, tokenizer, ingredients,name,vectorstore)
        #print("\n=== Complete Recipe Information ===")
        #print("Original Recipe:", generated_recipe['original_recipe'])
        #print("\nSubstitutions Made:", generated_recipe['substitutions'])
        #print("\nModified Recipe:", generated_recipe['modified_recipe'])
        recipe_dict = {
            "name": name,
            "ingredients": ingredients,
            "modified": generated_recipe['modified_recipe'],
            "reference_instructions": reference_instructions

        }
        recipes.append(recipe_dict)
        if len(recipes) % 50 == 0:
            temp_file = f"generated_recipes_temp_{len(recipes)}.json"
            with open(temp_file, "w") as f:
                json.dump(recipes, f, indent=4)
            print(f"\nProgress saved to {temp_file}")

    output_file = "generated_recipes_500.json"
    with open(output_file, "w") as f:
        json.dump(recipes, f, indent=4)

    print(f"\nAll recipes generated and saved to {output_file}")

if __name__ == "__main__":
    main()

Using device: cuda
Loading model and tokenizer...


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

light rum, currants, cooked white rice, milk, coconut milk, turbinado sugar, crystallized ginger, salt, fresh mango, papaya, shredded coconut

=== Allergen Substitutions Made ===
Replaced 'salt' with 'coconut yogurt'

Generating recipe for: catfish or any firm white fish with pecan sauce


100%|█████████▉| 499/500 [1:56:47<00:14, 14.31s/it]

catfish fillets, milk, pecans, butter, green onion, all-purpose flour, paprika, salt, garlic powder, onion powder, ground red pepper, white pepper, black pepper, dried thyme, dried oregano, fresh lime juice

=== Allergen Substitutions Made ===
Replaced 'butter' with 'coconut oil'
Replaced 'paprika' with 'hearts of palm'
Replaced 'salt' with 'coconut yogurt'
Replaced 'black pepper' with 'hearts of palm'

Generating recipe for: rosemary polenta pound cake


100%|██████████| 500/500 [1:57:01<00:00, 14.04s/it]

buttermilk, polenta, flour, baking powder, baking soda, salt, fresh rosemary leaves, unsalted butter, sugar, eggs

Progress saved to generated_recipes_temp_500.json

All recipes generated and saved to generated_recipes_500.json
